In [8]:
import requests
import pandas as pd

from PIL import Image
import io
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [9]:
def available_datasets(base_url):
    """
    Lists available urls to requests different scans
    input: root url pointing to the scan tiles provider
    output: datasets ids
    """    
    band_key = "band_id"
    response = requests.get(base_url+"/datasets") 
    
    datasets = response.json()["datasets"]
    datasets_df = pd.DataFrame.from_dict(datasets).drop(columns=band_key).drop_duplicates()
    datasets_ids = datasets_df.apply(lambda p:"/".join(p),axis=1)
    
    return datasets_ids

class TerraScan():
    def __init__(self,base_url,scan_id):
        
        self.scan_id = scan_id
        self.base_url = base_url
        self.rgb_suffix ="/{z}/{x}/{y}.png?r=band2&g=band1&b=band0"
        self.metadata = self._get_metadata()
        self.preview = self._get_preview()
    
    def _get_metadata(self):
        url = self.base_url+"/metadata/"+self.scan_id+"/band0"
        response = requests.get(url).json()
        return response
    
    def _get_preview(self):
        url = self.base_url+"/singleband/"+self.scan_id+"/band0/preview.png?tile_size=[128,128]"
        bytes_image = requests.get(url).content
        image = Image.open(io.BytesIO(bytes_image))
        return image
    
    @property
    def png_tile(self):
        url = self.base_url+"/rgb/"+self.scan_id+self.rgb_suffix
        return url
    def __repr__(self):
        return f"TerraScan({self.scan_id})"

In [10]:
import plotly.graph_objects as go

def dev_graph(preview):
    
    # Create figure
    fig = go.Figure()
    x_dots = preview.width*np.random.rand(10)*0.8
    y_dots = preview.height*np.random.rand(10)*0.8
    # Add trace
    fig.add_trace(
        go.Scatter(x=x_dots, y=y_dots,mode='markers',
                  marker=dict(color=np.random.randn(10),
                              colorscale='Viridis',
                              line_width=1,
                              size=7))
    )

    # Add images
    fig.add_layout_image(
            dict(
                source=preview,
                xref="x",
                yref="y",
                x=0,
                y=preview.height,
                sizex=preview.width,
                sizey=preview.height,
                layer="below")
    )

    # Set templates

    fig.update_layout(
        template="plotly_white",
        autosize=False,
        height=preview.height,
        width=preview.width,
        margin=dict(r=0, l=0, b=0, t=0))
    fig.update_xaxes(showgrid=False,visible=False,range=[0, preview.width])
    fig.update_yaxes(showgrid=False,visible=False,range=[0, preview.height])

    return fig

In [11]:
def build_map_dropdown():
    layout = dbc.InputGroup([
                dbc.InputGroupAddon("Tissue slice id", addon_type="prepend"),
                dbc.Select(
                        id='map-dropdown',
                        options=[{'label': scan_id, 'value': scan_id} for scan_id,_ in datasets.items()],
                        value=list(datasets.keys())[0]
                        ),
                
    ])
    
    return layout

In [ ]:
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_leaflet as dl
import dash
from dash.dependencies import Input, Output, State
import json


attribution = 'Nanostring DSP'

base_url = "http://localhost:5000"
datasets_ids = available_datasets(base_url)
datasets = {data_id:TerraScan(base_url,data_id) for data_id in datasets_ids}

# Create app.
app = dash.Dash(external_stylesheets=[dbc.themes.SANDSTONE])
app.layout = html.Div([
    
    html.Div([
        dl.Map(id='map',zoom=11,minZoom=6,
               style={'width': '100%', 'height': '70vh'}
              ),
        dcc.Graph(id='preview-graph',config = {'displayModeBar': False}
                 ),
    ],className='col-md-6'),
    
    html.Div([
        build_map_dropdown(),
        dbc.Tabs(id='aggregation-tab',
                 active_tab='global',
                 children=[
                          dbc.Tab(label="Global", tab_id="global"),
                          dbc.Tab(label="ROI", tab_id="roi"),
                 ]),
        html.Div(id='tab-content')
        ],className='col-md-6'),
    
    ],className='row')

@app.callback(
    [Output(component_id='map', component_property='children'),
     Output(component_id='map', component_property='center'),
     Output(component_id='preview-graph',component_property='figure')],
    Input(component_id='map-dropdown', component_property='value')
)
def switch_scan(scan_id):
    scan = datasets[scan_id]
    
    children = [dl.TileLayer(url=scan.png_tile, maxZoom=20, attribution=attribution)]
    x0,y0,x1,y1 = scan.metadata["bounds"]
    center = ( (y1-y0)/2 , (x1-x0)/2 ) 
    fig = dev_graph(scan.preview)
    
    return children,center,fig

@app.callback(
    Output('tab-content', 'children'),
    Input('aggregation-tab', 'active_tab'))
def switch_tabs(tab_value):
    if tab_value=='roi':
        return "Selected ROI"
    if tab_value=='global':
        return "Selected global"
    

@app.callback(
    [Output('map', 'center'),
    Output('map','zoom')],
    Input('preview-graph', 'clickData'),
    State('map-dropdown','value'))
def test_scatter(clickData,scan_id):
    if not clickData:
        return dash.no_update
    
    lon0,lat0,lon1,lat1 = datasets[scan_id].metadata["bounds"]
    lon_range = lon1-lon0
    lat_range = lat1-lat0
    
    coordinates = clickData["points"][0]
    x,y = coordinates["x"],coordinates["y"]
    lon = (x/128)*lon_range
    lat = (y/128)*lat_range
    center = (lat,lon)
    zoom = 12
    return center,zoom


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2021 15:12:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 15:12:26] "GET /_favicon.ico?v=1.16.1 HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 15:12:26] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 15:12:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 15:12:27] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [03/Apr/2021 15:12:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 15:12:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 15:13:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 15:13:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 15:13:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 15:13:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2021 15:13:09] "POST /_dash-update-component H

In [ ]:
a

In [1]:
import json

In [3]:
json.dumps({0: (255, 255, 255)})

'{"0": [255, 255, 255]}'

In [ ]:
from PIL import Image

In [6]:
base_url = "http://localhost:5000"
datasets_ids = available_datasets(base_url)
datasets = {data_id:TerraScan(base_url,data_id) for data_id in datasets_ids}

In [9]:
list(datasets.values())[0].preview.height

128

In [11]:
list(datasets.values())[0].preview.width

128

In [1]:
import dash_bootstrap_components as dbc

In [5]:
dir(dbc.themes)

['BOOTSTRAP',
 'CERULEAN',
 'COSMO',
 'CYBORG',
 'DARKLY',
 'FLATLY',
 'GRID',
 'JOURNAL',
 'LITERA',
 'LUMEN',
 'LUX',
 'MATERIA',
 'MINTY',
 'PULSE',
 'SANDSTONE',
 'SIMPLEX',
 'SKETCHY',
 'SLATE',
 'SOLAR',
 'SPACELAB',
 'SUPERHERO',
 'UNITED',
 'YETI',
 '_BOOTSWATCH_BASE',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [1]:
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_leaflet as dl
import dash
from dash.dependencies import Input, Output, State
import json

In [1]:
import dash_leaflet as dl

In [7]:
dir(dl.Map)

['REQUIRED',
 'UNDEFINED',
 '_REQUIRED',
 '_UNDEFINED',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_css_dist',
 '_get_set_or_delete',
 '_id_str',
 '_js_dist',
 '_traverse',
 '_traverse_ids',
 '_traverse_with_paths',
 'to_plotly_json']

Object `dl.set_map_bounds` not found.
